In [ ]:
from nn import Linear, Sigmoid, Relu, Softmax
from loss import MultiCrossEntropyLoss
from network import NeuralNet
import numpy as np
import logging
logging.basicConfig(format="%(asctime)s %(levelname)s: %(message)s",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
from keras.datasets import mnist


In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  '  + str(test_X.shape))
print('Y_test:  '  + str(test_y.shape))

In [ ]:
N_train = 1000
N_test = 10
train_X = train_X[0:N_train]
train_y = train_y[0:N_train]
test_X = test_X[0:N_test]
test_y = test_y[0:N_test]

In [ ]:
from matplotlib import pyplot
for i in range(9):  
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(train_X[i], cmap=pyplot.get_cmap('gray'))
    pyplot.show()


In [ ]:
train_y[0]

In [ ]:
def transfer_X(data_X):
    data_X_reshape = data_X.reshape(len(data_X), -1)
    data_X_reshape = data_X_reshape / 255.0
    return data_X_reshape

train_X_reshape = transfer_X(train_X)
print(train_X_reshape.shape)


In [ ]:
train_y.min(), train_y.max()

In [ ]:
def transfer_Y(data_y, n_classes = None):
    # one hot encoding
    # initial data_y is output of a class indices, e.g. [2, 1, 0, 9, ...]
    # convert it to one hot encoding, e.g. [[0,0,1,...], [0,1,0...], ...]
    nd = len(data_y)
    if n_classes is None:
        n_classes = data_y.max() + 1
    encoded_data_y = np.zeros((nd, n_classes), dtype=int)
    encoded_data_y[np.arange(nd), data_y] = 1
    return encoded_data_y

n_classes = 10
train_y_encoded = transfer_Y(train_y, n_classes)
train_y_encoded[0:3]


In [ ]:
n_nonzero = 0
for idx, val in enumerate(train_X_reshape[0]):
    if val > 0:
        n_nonzero += 1
        if n_nonzero <= 10:
            print(idx, val) 
n_nonzero

In [ ]:
nd_X, nd_y = len(train_X_reshape[0]), len(train_y_encoded[0])
print(f"Dimensions, X {nd_X}, y {nd_y}")
learning_rate = 0.001
net = NeuralNet()
net.add_layer(Linear(nd_X, 64, learning_rate))
net.add_layer(Relu())
net.add_layer(Linear(64, 24, learning_rate))
net.add_layer(Relu())
net.add_layer(Linear(24, nd_y, learning_rate))
net.add_layer(Softmax())

In [ ]:
net.loss = MultiCrossEntropyLoss()

In [ ]:
train_y_encoded.shape

In [ ]:
net.train(train_X_reshape, train_y_encoded, n_epoch=2)

In [ ]:
test_X_reshape = transfer_X(test_X)
print(test_X_reshape.shape)
test_y_encoded = transfer_Y(test_y, n_classes)

In [ ]:
predicted = net.test(test_X_reshape, test_y_encoded)
predicted.shape


In [ ]:
predicted[0]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, average_precision_score, recall_score, multilabel_confusion_matrix


In [ ]:
test_y_predicted = np.argmax(predicted, axis=1)
print(test_y_predicted[0])
test_y_predicted.shape

In [ ]:
cm = multilabel_confusion_matrix(test_y, test_y_predicted)
cm

In [ ]:
result = {
    "accuracy": accuracy_score(test_y, test_y_predicted),
    "precision": precision_score(test_y, test_y_predicted, average="micro"),
    "recall": recall_score(test_y, test_y_predicted, average="micro")
}
result